In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from modules.loader import ImmothepDataLoader
loader = ImmothepDataLoader()
del(ImmothepDataLoader)

from modules.csvLoader import csvLoader

DATASET_PATH = '../datas/RAW/'

%load_ext autoreload
%autoreload 2

## Téléchargement des données

In [2]:
loader.ensure_data_loaded()
del(loader)

Les fichiers sont correctement téléchargés


## Lecture des csv

In [3]:
valeurs_foncieres = pd.read_csv(DATASET_PATH + "valeursfoncieres-2019.txt", sep='|', decimal=',', encoding='utf-8')

del(DATASET_PATH)

# Observations sur le dataframe

## Les types de biens
```
Appartement
Dépendance
Maison
nan
Local industriel. commercial ou assimilé

nb local :	 2535791  soit	 100  % des locaux
nb appart :	 430975  soit	 17  % des locaux
nb maison :	 515398  soit	 20  % des locaux
nb dependance :	 312480  soit	 12  % des locaux
nb none_local :	 1182459  soit	 47  % des locaux
nb industriel :	 94479  soit	 4  % des locaux
```


In [4]:
for local in valeurs_foncieres['Type local'].drop_duplicates().values:
    print(local)
del(local)
print()
appartement = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Appartement']
maison = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Maison']
# dependance = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Dépendance']
# none_local = valeurs_foncieres[valeurs_foncieres['Type local'].isna()]
# industriel = valeurs_foncieres[valeurs_foncieres['Type local'] == 'Local industriel. commercial ou assimilé']

print('nb local :\t', len(valeurs_foncieres), '\tsoit\t', round(len(valeurs_foncieres)/len(valeurs_foncieres)*100), ' % des locaux' )
print('nb appart :\t', len(appartement), '\tsoit\t', round(len(appartement)/len(valeurs_foncieres)*100), ' % des locaux' )
print('nb maison :\t', len(maison), '\tsoit\t', round(len(maison)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb dependance :\t', len(dependance), '\tsoit\t', round(len(dependance)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb none_local :\t', len(none_local), '\tsoit\t', round(len(none_local)/len(valeurs_foncieres)*100), ' % des locaux' )
# print('nb industriel :\t', len(industriel), '\tsoit\t', round(len(industriel)/len(valeurs_foncieres)*100), ' % des locaux' )

del(valeurs_foncieres)

Appartement
Dépendance
Maison
nan
Local industriel. commercial ou assimilé

nb local :	 2535791 	soit	 100  % des locaux
nb appart :	 430975 	soit	 17  % des locaux
nb maison :	 515398 	soit	 20  % des locaux


# Maison

## Filtres sur les maisons

Aucune de ces maisons n'est souhaitée dans les données du modèle IA
- nb maison non bati 	 10 	soit	 0.0  % des maisons
- nb maison sans terrain 	 22207 	soit	 4.3  % des maisons
- nb maison sans valeur 	 1419 	soit	 0.3  % des maisons
- nb maison sans piece 	 10 	soit	 0.0  % des maisons
- nb maison sans code postal 	 45 	soit	 0.0  % des maisons
- maisons_filtre == 491748

In [5]:
cols = ['Surface reelle bati', 'Surface terrain', 'Valeur fonciere', 'Nombre pieces principales', 'Code postal']

for col in cols:
    df_vide = maison[maison[col].isna()]
    print('nb ' + col + ' \t', len(df_vide), '\tsoit\t', round(len(df_vide)/len(maison)*100, 1), ' % des maisons' )

maison_filtre = maison.dropna(how='any', subset=cols)

del(df_vide, col, cols)
del(maison)

print('maison_filtre ==', len(maison_filtre))

nb Surface reelle bati 	 10 	soit	 0.0  % des maisons
nb Surface terrain 	 22207 	soit	 4.3  % des maisons
nb Valeur fonciere 	 1419 	soit	 0.3  % des maisons
nb Nombre pieces principales 	 10 	soit	 0.0  % des maisons
nb Code postal 	 45 	soit	 0.0  % des maisons
maison_filtre == 491748


## Observation du contenu des colonnes

Dans un objectif de performance, est-il nécessaire de toutes les conserver ?


In [6]:
maison_filtre = maison_filtre.dropna(axis=1, how='all')

print('\nSuppression effectuée !')


Suppression effectuée !


In [7]:
print('Pourcentage valeur non vide par colonne')
for col in maison_filtre.columns:
    print( col, ' :\t', 100 - len(maison_filtre[maison_filtre[col].isna()]) / len(maison_filtre) * 100, ' %')

Pourcentage valeur non vide par colonne
No disposition  :	 100.0  %
Date mutation  :	 100.0  %
Nature mutation  :	 100.0  %
Valeur fonciere  :	 100.0  %
No voie  :	 99.63761926840577  %
B/T/Q  :	 7.106078723248487  %
Type de voie  :	 80.96565720653669  %
Code voie  :	 100.0  %
Voie  :	 99.99247582094894  %
Code postal  :	 100.0  %
Commune  :	 100.0  %
Code departement  :	 100.0  %
Code commune  :	 100.0  %
Prefixe de section  :	 3.76819021124642  %
Section  :	 99.99694965714146  %
No plan  :	 100.0  %
1er lot  :	 0.27798791250803845  %
Surface Carrez du 1er lot  :	 0.19176488770671085  %
Nombre de lots  :	 100.0  %
Code type local  :	 100.0  %
Type local  :	 100.0  %
Surface reelle bati  :	 100.0  %
Nombre pieces principales  :	 100.0  %
Nature culture  :	 100.0  %
Nature culture speciale  :	 3.278508504355898  %
Surface terrain  :	 100.0  %


In [8]:
csvLoader.save(maison_filtre, 'maison', save_index=False)

# Appartement

## Filtres sur les appartements

In [9]:
cols = ['Surface reelle bati', 'Valeur fonciere', 'Nombre pieces principales', 'Code postal']

for col in cols:
    df_vide = appartement[appartement[col].isna()]
    print('nb ' + col + ' \t', len(df_vide), '\tsoit\t', round(len(df_vide)/len(appartement)*100, 1), ' % des appartements' )

appartement_filtre = appartement.dropna(how='any', subset=cols)

del(df_vide, col, cols)
del(appartement)

print('appartement_filtre ==', len(appartement_filtre))

nb Surface reelle bati 	 11 	soit	 0.0  % des appartements
nb Valeur fonciere 	 2346 	soit	 0.5  % des appartements
nb Nombre pieces principales 	 11 	soit	 0.0  % des appartements
nb Code postal 	 9 	soit	 0.0  % des appartements
appartement_filtre == 428609


## Observation du contenu des colonnes

Dans un objectif de performance, est-il nécessaire de toutes les conserver ?

In [10]:
appartement_filtre = appartement_filtre.dropna(axis=1, how='all')

print('\nSuppression effectuée !')


Suppression effectuée !


In [11]:
print('Pourcentage valeur non vide par colonne')
for col in appartement_filtre.columns:
    print(col, ' :\t', 
        len(appartement_filtre) - len(appartement_filtre[appartement_filtre[col].isna()]), 
        '\t\t', 
        100 - len(appartement_filtre[appartement_filtre[col].isna()]) / len(appartement_filtre) * 100, 
        ' %'
    )

del(col)

Pourcentage valeur non vide par colonne
No disposition  :	 428609 		 100.0  %
Date mutation  :	 428609 		 100.0  %
Nature mutation  :	 428609 		 100.0  %
Valeur fonciere  :	 428609 		 100.0  %
No voie  :	 425109 		 99.18340492150188  %
B/T/Q  :	 25444 		 5.936412907801753  %
Type de voie  :	 415941 		 97.04439244159596  %
Code voie  :	 428609 		 100.0  %
Voie  :	 428607 		 99.99953337424085  %
Code postal  :	 428609 		 100.0  %
Commune  :	 428609 		 100.0  %
Code departement  :	 428609 		 100.0  %
Code commune  :	 428609 		 100.0  %
Prefixe de section  :	 23143 		 5.399559971909127  %
Section  :	 428598 		 99.99743355832472  %
No plan  :	 428609 		 100.0  %
1er lot  :	 320057 		 74.67342029682065  %
Surface Carrez du 1er lot  :	 176937 		 41.28168097263474  %
2eme lot  :	 146467 		 34.17263753210969  %
Surface Carrez du 2eme lot  :	 50561 		 11.796532503983812  %
3eme lot  :	 19337 		 4.51157115226232  %
Surface Carrez du 3eme lot  :	 4114 		 0.959849186554635  %
4eme lot  :	 5552 		 1

In [12]:
csvLoader.save(appartement_filtre, 'appartement', save_index=False)